In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib

from scipy.linalg import expm, null_space

plt.style.use("ggplot")
plt.rcParams["font.family"] = "sans-serif"
sns.set_palette("dark")
sns.set_context("talk")

import numpy as np

from IPython.display import HTML, display, SVG

import pydot
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder, BasicVector,
                        Parser, Saturation, Simulator, PlanarSceneGraphVisualizer, 
                        LinearQuadraticRegulator, AbstractValue, MeshcatVisualizer, 
                        StartMeshcat, WrapToSystem, SceneGraph, DiscreteTimeLinearQuadraticRegulator,
                        JacobianWrtVariable, eq, SnoptSolver)    

from pydrake.systems.primitives import LogVectorOutput, ConstantVectorSource, ZeroOrderHold
from pydrake.multibody.inverse_kinematics import PositionConstraint, AddUnitQuaternionConstraintOnPlant

from pydrake.multibody.plant import ContactResults, CoulombFriction
from pydrake.autodiffutils import AutoDiffXd, InitializeAutoDiff, ExtractGradient, ExtractValue
from pydrake.math import RollPitchYaw_, RotationMatrix_, RigidTransform_
from underactuated.meshcat_cpp_utils import MeshcatSliders, MeshcatJointSliders


Using matplotlib backend: TkAgg


In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
time_step = 1/1000.
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
body = Parser(plant).AddModelFromFile("Double_Leg/Double_Leg_Collisions.urdf")
plant.Finalize()

meshcat.Delete()
vis = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
context = plant.CreateDefaultContext()

num_pos = plant.num_positions()
num_vel = plant.num_velocities()
num_act = plant.num_actuators()
num_disc_states = context.num_total_states() - context.num_continuous_states()
fb_num_pos = 7
fb_num_vel = 6
n = 6

x0 = np.zeros(num_disc_states)
x0[0], x0[6] = 1, 0.4

plant.SetPositionsAndVelocities(context, x0)
plant.get_actuation_input_port().FixValue(context, np.zeros(num_act))

def sethome(plant, context, ad=1.):

    shaft_roll = 0.3
    hip_pitch = 0.3
    knee_pitch = 0.6
    foot_pitch = 0.3

    plant.GetJointByName("shaft_roll").set_angle(context, shaft_roll)
    plant.GetJointByName("Hip_Pitch_R").set_angle(context, hip_pitch)
    plant.GetJointByName("Hip_Pitch_L").set_angle(context, hip_pitch)
    plant.GetJointByName("knee_R").set_angle(context, -knee_pitch)
    plant.GetJointByName("knee_L").set_angle(context, -knee_pitch)
    plant.GetJointByName("foot_leg_pitch_R").set_angle(context, foot_pitch)
    plant.GetJointByName("foot_leg_pitch_L").set_angle(context, -foot_pitch)

    foot_pos = plant.GetBodyByName("collision_dummy_L_C").EvalPoseInWorld(context).GetAsMatrix34()[:, -1].reshape(3)
    body_pos = plant.GetBodyByName("base_link").EvalPoseInWorld(context).GetAsMatrix34()[:, -1].reshape(3)

    body_pos[2] = body_pos[2] - foot_pos[2] + 0.001 # Add radius of collision sphere
    body_pos[1] = -foot_pos[1]

    if np.array([ad]).dtype == float:
        transform = RigidTransform_[float](body_pos)
    else:
        transform = RigidTransform_[AutoDiffXd](body_pos)

    plant.SetFreeBodyPose(context, plant.GetBodyByName("base_link"), transform)


sethome(plant, context)

x0 = plant.GetPositionsAndVelocities(context)

In [4]:
np.random.seed(5)

# Change between 0 input and random input
generic_input = builder.AddSystem(ConstantVectorSource(np.random.randn(num_act)))

print(np.max(np.random.randn(num_act)))

# generic_input = builder.AddSystem(ConstantVectorSource(np.zeros(num_act)))
builder.Connect(generic_input.get_output_port(), plant.get_actuation_input_port())
logger_states = LogVectorOutput(plant.get_state_output_port(), builder, publish_period=1/1000)
diagram = builder.Build()

simulator = Simulator(diagram)
sim_context = simulator.get_mutable_context()

x0[6] += 0.3

sim_context.SetTime(0.)
sim_context.SetDiscreteState(x0)
vis.StartRecording()
simulator.AdvanceTo(2)
vis.StopRecording()
vis.PublishRecording()

meshcat.DeleteAddedControls()

state_log = logger_states.FindLog(sim_context).data()
t = np.arange(0, len(state_log[0, :]))

fig, ax = plt.subplots()
ax.plot(t, state_log[0, :], label="q0")
ax.plot(t, state_log[1, :], label="q1")
ax.plot(t, state_log[2, :], label="q2")
ax.plot(t, state_log[3, :], label="q3")
ax.legend()

plt.show()



1.8573310072313118
